In [25]:
import pickle as pkl
import torch

components_list = [
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningEmpty',
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningError',
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningNULL'
]
prob_postfix_list = [
    '_joint0sh_beams_log_prob.pkl',
    '_joint1sh1_beams_log_prob.pkl',
    '_joint1sh2_beams_log_prob.pkl',
    '_joint1sh3_beams_log_prob.pkl',
    '_joint1sh4_beams_log_prob.pkl',
    '_joint1sh5_beams_log_prob.pkl'
]
output_postfix = '_joint6comp_beams_log_prob.pkl'

for comp_candidates in components_list:
    print(comp_candidates)
    log_sum = None
    for postfix in prob_postfix_list:
        filename= comp_candidates + postfix
        with open(filename , 'rb' ) as f:
            prob_list = pkl.load(f)
            # prob_list = torch.load(f, map_location=torch.device('cpu'), weights_only=False)
        print( prob_list[0:4] )
        prob_tensor = torch.stack(prob_list)
        if log_sum is None:
            log_sum = prob_tensor
        else:
            # print(f'selection_score_with_other_response:{selection_score_with_other_response} --> {selection_score_with_other_response.dtype}')
            # print(f'selection_score:{selection_score} --> {selection_score.dtype}')
            alpha = torch.maximum( prob_tensor , log_sum )
            beta = torch.minimum( prob_tensor , log_sum )
            log_sum = alpha + torch.log1p(torch.exp(beta-alpha))
    log_sum_list = log_sum.tolist()
    with open(comp_candidates+output_postfix , 'wb') as f:
        pkl.dump(log_sum_list, f)
        

./outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningEmpty
[tensor(-5.5477), tensor(-3.0003), tensor(-3.0134), tensor(-4.0511)]
[tensor(-5.6259), tensor(-3.0719), tensor(-3.0888), tensor(-4.1120)]
[tensor(-5.2717), tensor(-2.6832), tensor(-2.6922), tensor(-3.7161)]
[tensor(-5.5276), tensor(-2.8979), tensor(-2.9253), tensor(-3.9666)]
[tensor(-5.3546), tensor(-2.7303), tensor(-2.7446), tensor(-3.7522)]
[tensor(-5.5138), tensor(-2.9950), tensor(-3.0019), tensor(-4.0470)]
./outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningError
[tensor(-4.3410), tensor(-4.1077), tensor(-7.8504), tensor(-3.0475)]
[tensor(-4.1401), tensor(-3.9294), tensor(-7.7465), tensor(-2.8620)]
[tensor(-4.0389), tensor(-3.8317), tensor(-7.6233), tensor(-2.7040)]
[tensor(-4.0657), tensor(-3.9038), tensor(-7.5422), tensor(-2.7624)]
[tensor(-4.2351), tensor(-4.0185), tensor(-7.6680), tensor(-2.9239)]
[tensor(-4.0818), tensor(-3.8854), tensor(-7.6906), tensor(-2.7465)]
./output

In [41]:
import math
num_beam = 16 # We should not forget to set this parameter correctly!!
new_candid_prob_list = []
new_output_prefix = '_a1b1Joint6comp_beams_log_prob.pkl'
for beam_index in range(0,1534*num_beam,num_beam):
    candid_prob_list = []
    for idx , comp_candidates in enumerate(components_list):
        
        if idx <= len(new_candid_prob_list):
            new_candid_prob_list.append([])
            
        filename= comp_candidates + output_postfix
        with open(filename , 'rb' ) as f:
            prob_list = pkl.load(f)
        candid_prob_list.extend(prob_list[beam_index:beam_index+num_beam])
        
    candid_prob_list_min = min(candid_prob_list)
    candid_prob_list_max = max(candid_prob_list)
    candid_prob_list_scaled = [(x - candid_prob_list_min) / (candid_prob_list_max - candid_prob_list_min) for x in candid_prob_list]
    
    # Step 2: Apply exponentiation to amplify differences
    # alpha = 1  # You can experiment with different values #It kinda give the original value
    # beta = .087 #It kinda give the original value
    # gama = 0.275 #It kinda give the original value
    alpha = 1  
    beta = 1 
    gama = 0 
    p_transformed = [beta * x ** alpha + gama for x in candid_prob_list_scaled]
    for i in range(0, len(p_transformed), num_beam):
        new_candid_prob_list[ math.ceil(i/num_beam) ].extend(p_transformed[i:i+num_beam])

for prob_list, comp_candidates in zip(new_candid_prob_list, components_list):
    with open( comp_candidates + new_output_prefix , 'wb') as f:
        pkl.dump(prob_list, f)



In [40]:
#The code to put the refined queries back into their place
num_beam = 16 # We should not forget to set this parameter correctly!!
refined_candidates_files = [
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningEmpty.pkl',
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningError.pkl',
    './outputs/raw/output_01shot_6in00-5shotComps_16samples_SFTCodeS-15b_RefiningNULL.pkl'
]
original_components = [
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-15b.pkl',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_1Of0-5shot.pkl',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_2Of0-5shot.pkl',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_3Of0-5shot.pkl',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_4Of0-5shot.pkl',
    './outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_5Of0-5shot.pkl'
]
probability_postfix = '_joint6comp_beams_log_prob.pkl'

new_components_postfix = '_refinedNull'

old_candids = []
old_probs = []
#Loading all the files in lists
for orig_comp in original_components:
    with open(orig_comp , 'rb') as f:
        old_candids.append(pkl.load(f))
    with open( orig_comp[:-4] + probability_postfix , 'rb') as f:
        old_probs.append(pkl.load(f))
        
refined_candid_file = refined_candidates_files[ 2 ]
with open( refined_candid_file , 'rb') as f:
    refined_candids = pkl.load(f)
with open( refined_candid_file[:-4]+probability_postfix , 'rb' ) as f:
    refined_probs = pkl.load(f)


last_prob_idx = 0
for idx , refine_dict in enumerate(refined_candids):
    for key in refine_dict.keys():
        start_in_orig_comps = idx*num_beam
        for comp_idx , comp_old_candids in enumerate(old_candids):
            for beam_idx , candid in enumerate(comp_old_candids[ start_in_orig_comps : start_in_orig_comps + num_beam ]):
                if key == candid:
                    old_candids[comp_idx][start_in_orig_comps + beam_idx] = refine_dict[key]
                    old_probs[comp_idx][start_in_orig_comps + beam_idx] = refined_probs[last_prob_idx]
        last_prob_idx += 1

print('Saving the results in:')
components_list = []
output_postfix = probability_postfix #so that we can scale the probabilities in the above cell
for idx , (candids , probs) in enumerate(zip(old_candids,old_probs)):
    comp_candid_filename = original_components[idx][:-4] + new_components_postfix + '.pkl'
    comp_prob_filename = original_components[idx][:-4] + new_components_postfix + probability_postfix
    with open(comp_candid_filename , 'wb') as f:
        pkl.dump(candids , f)
        print(comp_candid_filename)
        components_list.append(comp_candid_filename[:-4]) #so that we can scale the probabilities in the above cell
    with open(comp_prob_filename , 'wb') as f:
        pkl.dump(probs , f)
        print(comp_prob_filename)
    

Saving the results in:
./outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-15b_refinedNull.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_0shot_SFTCodeS-15b_refinedNull_joint6comp_beams_log_prob.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_1Of0-5shot_refinedNull.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_1Of0-5shot_refinedNull_joint6comp_beams_log_prob.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_2Of0-5shot_refinedNull.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_2Of0-5shot_refinedNull_joint6comp_beams_log_prob.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_3Of0-5shot_refinedNull.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_3Of0-5shot_refinedNull_joint6comp_beams_log_prob.pkl
./outputs/raw/output_CodeS_candid16_bird_wEvidence_1shot_SFTCodeS-15b_4Of0-5shot_refinedNull.pkl
./outputs/raw/output_C